In [1]:
# save the final model to file
%pip install tensorflow keras
from keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dense
from keras.layers import Flatten
from tensorflow.keras.optimizers import SGD

     |████████████████████████████████| 497.6 MB 363 bytes/s a 0:00:01                    | 22.1 MB 173 kB/s eta 0:45:42     |█▌                              | 23.0 MB 91 kB/s eta 1:26:02.1 MB 91 kB/s eta 1:25:50.9 MB 91 kB/s eta 1:25:09.4 MB 91 kB/s eta 1:24:41388:00     |██▍                             | 37.7 MB 51 kB/s eta 2:27:33 MB 14.2 MB/s eta 0:00:31 |████▉                           | 75.8 MB 12.0 MB/s eta 0:00:36                    | 78.6 MB 3.8 MB/s eta 0:01:49     |█████▊                          | 88.8 MB 3.8 MB/s eta 0:01:47     |██████▌                         | 101.2 MB 8.6 MB/s eta 0:00:4766                | 122.5 MB 7.4 MB/s eta 0:00:51                | 123.8 MB 7.4 MB/s eta 0:00:51172 MB 50 kB/s eta 1:54:28MB/s eta 0:00:27     |███████████                     | 171.7 MB 12.6 MB/s eta 0:00:26      | 175.5 MB 366 kB/s eta 0:14:39      | 176.5 MB 366 kB/s eta 0:14:36��██████▍                    | 177.3 MB 366 kB/s eta 0:14:34      | 178.0 MB 366 kB/s eta 0:14:32      | 1

2022-02-13 18:28:25.844701: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-02-13 18:28:25.844776: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
# load train and test Cifar10 Dataset
def load_dataset():
  # load dataset
  (trainX, trainY), (testX, testY) = cifar10.load_data()
 
  # reshape dataset to have a single channel
  # trainX = trainX.reshape((trainX.shape[0], 32, 32, 1))
  # testX = testX.reshape((testX.shape[0], 32, 32, 1))
	# one hot encode target values
  trainY = to_categorical(trainY)
  testY = to_categorical(testY)
  return trainX, trainY, testX, testY
  
load_dataset()

170508288/170498071 [==============================] - 21s 0us/step


(array([[[[ 59,  62,  63],
          [ 43,  46,  45],
          [ 50,  48,  43],
          ...,
          [158, 132, 108],
          [152, 125, 102],
          [148, 124, 103]],
 
         [[ 16,  20,  20],
          [  0,   0,   0],
          [ 18,   8,   0],
          ...,
          [123,  88,  55],
          [119,  83,  50],
          [122,  87,  57]],
 
         [[ 25,  24,  21],
          [ 16,   7,   0],
          [ 49,  27,   8],
          ...,
          [118,  84,  50],
          [120,  84,  50],
          [109,  73,  42]],
 
         ...,
 
         [[208, 170,  96],
          [201, 153,  34],
          [198, 161,  26],
          ...,
          [160, 133,  70],
          [ 56,  31,   7],
          [ 53,  34,  20]],
 
         [[180, 139,  96],
          [173, 123,  42],
          [186, 144,  30],
          ...,
          [184, 148,  94],
          [ 97,  62,  34],
          [ 83,  53,  34]],
 
         [[177, 144, 116],
          [168, 129,  94],
          [179, 142,  87],
   

In [3]:
# scale pixels
def prep_pixels(train, test):
	# convert from integers to floats
	train_norm = train.astype('float32')
	test_norm = test.astype('float32')
	# normalize to range 0-1
	train_norm = train_norm / 255.0
	test_norm = test_norm / 255.0
	# return normalized images
	return train_norm, test_norm

trainX, trainY, testX, testY = load_dataset()
prep_pixels(trainX, trainY)

(array([[[[0.23137255, 0.24313726, 0.24705882],
          [0.16862746, 0.18039216, 0.1764706 ],
          [0.19607843, 0.1882353 , 0.16862746],
          ...,
          [0.61960787, 0.5176471 , 0.42352942],
          [0.59607846, 0.49019608, 0.4       ],
          [0.5803922 , 0.4862745 , 0.40392157]],
 
         [[0.0627451 , 0.07843138, 0.07843138],
          [0.        , 0.        , 0.        ],
          [0.07058824, 0.03137255, 0.        ],
          ...,
          [0.48235294, 0.34509805, 0.21568628],
          [0.46666667, 0.3254902 , 0.19607843],
          [0.47843137, 0.34117648, 0.22352941]],
 
         [[0.09803922, 0.09411765, 0.08235294],
          [0.0627451 , 0.02745098, 0.        ],
          [0.19215687, 0.10588235, 0.03137255],
          ...,
          [0.4627451 , 0.32941177, 0.19607843],
          [0.47058824, 0.32941177, 0.19607843],
          [0.42745098, 0.28627452, 0.16470589]],
 
         ...,
 
         [[0.8156863 , 0.6666667 , 0.3764706 ],
          [0.78823

In [4]:
def define_model():
    model = Sequential([
          # First Convolution Layer
         Conv2D(32, (3,3), padding="same", activation='relu', kernel_initializer='he_uniform'),
         Conv2D(32, (3,3), padding="same", activation='relu', kernel_initializer='he_uniform'),
         MaxPooling2D(pool_size=(2, 2)),

         #Second Convolution Layer
         Conv2D(64, (3,3), padding="same", activation='relu', kernel_initializer='he_uniform'),
         Conv2D(64, (3,3), padding="same", activation='relu', kernel_initializer='he_uniform'),
         MaxPooling2D(pool_size=(2, 2)),

        #Third Convolution Layer
         Conv2D(128, (3,3), padding="same", activation='relu', kernel_initializer='he_uniform'),
         Conv2D(128, (3,3), padding="same", activation='relu', kernel_initializer='he_uniform'),
         MaxPooling2D(pool_size=(2, 2)),

        #  Fully Connected Layer
        Flatten(),
        Dense(100, activation='relu', kernel_initializer='he_uniform'),
        Dense(10, activation='softmax')])
 
 

    opt = SGD(learning_rate=0.001, momentum=0.9)
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

  
    return model
    
define_model()

2022-02-13 18:29:32.121352: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-02-13 18:29:32.130101: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-02-13 18:29:32.131432: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (DESKTOP-F5IN9DC): /proc/driver/nvidia/version does not exist
2022-02-13 18:29:32.147295: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [ ]:
# run the test harness for evaluating a model
def run_test_harness():
    # load dataset
    trainX, trainY, testX, testY = load_dataset()
    # prepare pixel data
    trainX, testX = prep_pixels(trainX, testX)
    # define model
    model = define_model()
    # fit model
    model.fit(trainX, trainY, epochs=10, batch_size=32, verbose=0)
    # save model
    model.save('cifar_final_model.h5')

run_test_harness()